<a href="https://colab.research.google.com/github/rajeshkpandey/AWESOME-FER/blob/master/Job_Title_Prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
print(os.listdir("./"))

import pandas as pd
import numpy as np
from scipy.stats import randint
import seaborn as sns # used for plot interactive graph. 
import matplotlib.pyplot as plt
import seaborn as sns
from io import StringIO
import string
import nltk
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from nltk.stem import WordNetLemmatizer
import re 
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_selection import chi2
from IPython.display import display
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import LinearSVC
from sklearn.model_selection import cross_val_score
from sklearn.metrics import confusion_matrix
from sklearn import metrics

['.config', 'test.csv', '.ipynb_checkpoints', 'sample_data']


In [2]:
df = pd.read_csv('train.csv')
df.head()
test_df = pd.read_csv('test.csv')
test_df.head()

,Job ID,Job Description,Title
0,JOB0000,TeamSoft is seeing an IT Support Specialist to...,NaN
1,JOB0001,The Wisconsin State Journal is seeking a flexi...,NaN
2,JOB0002,Report this job About the Job DePuy Synthes Co...,NaN
3,JOB0003,Why Join Altec? If you’re considering a career...,NaN
4,JOB0004,Position ID# 76162 # Positions 1 State CT C...,NaN


In [3]:
df.shape
test_df.shape

(2000, 3)

In [ ]:
df.tail()
test_df.tail()

In [5]:
nltk.download('wordnet')
nltk.download('stopwords')
#cleaning training dataset
for i in range(len(df)):
  df['Job Description'][i] = df['Job Description'][i].lower()
  df['Job Description'][i]="".join(char for char in df['Job Description'][i] if char not in string.punctuation)
  ps = PorterStemmer()
  wml = WordNetLemmatizer()
  df['Job Description'][i]=ps.stem(df['Job Description'][i])
  df['Job Description'][i]=wml.lemmatize(df['Job Description'][i])
  df['Job Description'][i]=df['Job Description'][i].split()
  filter_words = []
  Stopwords = set(stopwords.words('english'))
  for word in df['Job Description'][i]:
    if word not in Stopwords:
         filter_words.append(word)
  df['Job Description'][i] = filter_words 

#cleaning testing set
for i in range(len(test_df)):
  test_df['Job Description'][i] = test_df['Job Description'][i].lower()
  test_df['Job Description'][i]="".join(char for char in test_df['Job Description'][i] if char not in string.punctuation)
  ps = PorterStemmer()
  wml = WordNetLemmatizer()
  test_df['Job Description'][i]=ps.stem(test_df['Job Description'][i])
  test_df['Job Description'][i]=wml.lemmatize(test_df['Job Description'][i])
  test_df['Job Description'][i]=test_df['Job Description'][i].split()
  filter_words = []
  Stopwords = set(stopwords.words('english'))
  for word in test_df['Job Description'][i]:
    if word not in Stopwords:
         filter_words.append(word)
  test_df['Job Description'][i] = filter_words 

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:9: Se

In [6]:
df['Job Description'][0]

['teamsoft',
 'seeing',
 'support',
 'specialist',
 'join',
 'client',
 'madison',
 'wi',
 'ideal',
 'candidate',
 'must',
 'least',
 '6',
 'years',
 'experience',
 'field',
 'need',
 'familiar',
 'variety',
 'fields',
 'concepts',
 'practices',
 'procedures',
 'position',
 'relies',
 'extensive',
 'experience',
 'judgment',
 'plan',
 'accomplish',
 'goals',
 'required',
 'skills•',
 'call',
 'tracking',
 'software•',
 'phone',
 'based',
 'technical',
 'support•',
 'problem',
 'documentation',
 'communication•',
 'remote',
 'desktop',
 'management',
 'tools•',
 'respond',
 'customer',
 'requests•',
 'general',
 'understanding',
 'landesk•',
 'microsoft',
 'office',
 '2007',
 'suitefind',
 'teamsoft',
 'madison',
 'areas',
 'technology',
 'leader',
 '200',
 'consultants',
 'owned',
 'operated',
 'managed',
 'consultants',
 'teamsoft',
 'clear',
 'choice',
 'career',
 'join',
 'teamsoft',
 'family',
 'todaydid',
 'know•',
 'teamsoft',
 'offers',
 'salaried',
 'options',
 'many',
 'positi

In [7]:
# Create a new dataframe with two columns
df1 = df[['Job Description', 'Title']].copy()

# Remove missing values (NaN)
df1 = df1[pd.notnull(df1['Job Description'])]

# Renaming job description column for a simpler name
df1.columns = ['Job_Description', 'Title'] 

df1.shape

(2000, 2)

In [8]:
df1.tail()

,Job_Description,Title
1995,"[general, descriptionresponsible, ensuring, pr...",NaN
1996,"[job, id, 94448nonexempt, oldcastle, precast, ...",NaN
1997,"[general, descriptionis, responsible, phases, ...",NaN
1998,"[report, job, job, skilled, nursing, facility,...",NaN
1999,"[job, id, 99428, georgia, masonry, supply, div...",NaN


In [9]:
# Percentage of Job Description with text
total = df1['Job_Description'].notnull().sum()
round((total/len(df)*100),1)

100.0

In [10]:
pd.DataFrame(df.Title.unique()).values

array([[nan]])

In [11]:
len(df.Title.unique())

1

In [12]:
df2 = df1.sample(10000, random_state=1).copy()

ValueError: ignored

In [ ]:
# Create a new column 'category_id' with encoded categories 
df2['category_id'] = df2['Title'].factorize()[0]
category_id_df = df2[['Title', 'category_id']].drop_duplicates()


# Dictionaries for future use
category_to_id = dict(category_id_df.values)
id_to_category = dict(category_id_df[['category_id', 'Title']].values)

# New dataframe
df2.head()

In [ ]:
df2.tail()

In [ ]:
tfidf = TfidfVectorizer(sublinear_tf=True, min_df=5,
                        ngram_range=(1, 2), 
                        stop_words='english')

# We transform each Job Description into a vector
features = tfidf.fit_transform(df2.Job_Description).toarray()

labels = df2.category_id

print("Each of the %d job descriptions is represented by %d features (TF-IDF score of unigrams and bigrams)" %(features.shape))

In [ ]:
X = df2['Job_Description'][:1000] # Collection of Job descriptions
y = df2['Title'][:1000] # Target or the labels we want to predict

X_train, X_test, y_train, y_test = train_test_split(X, y, 
                                                    test_size=0.25,
                                                    random_state = 0)

In [ ]:
models = [
    RandomForestClassifier(n_estimators=50, max_depth=5, random_state=0),
    LinearSVC(),
    MultinomialNB(),
    LogisticRegression(random_state=0),
]

# 5 Cross-validation
CV = 5
cv_df = pd.DataFrame(index=range(CV * len(models)))

entries = []
for model in models:
  model_name = model.__class__.__name__
  accuracies = cross_val_score(model, features, labels, scoring='accuracy', cv=CV)
  for fold_idx, accuracy in enumerate(accuracies):
    entries.append((model_name, fold_idx, accuracy))
    
cv_df = pd.DataFrame(entries, columns=['model_name', 'fold_idx', 'accuracy'])